In [1]:
import os

In [2]:
%pwd

'd:\\myprojects\\Chicken_disease_classification\\Chicken-disease-classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\myprojects\\Chicken_disease_classification\\Chicken-disease-classification'

In [ ]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from chicken_disease_prediction.constants import *
from chicken_disease_prediction.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
          config=self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )
          return data_ingestion_config
            
    

In [8]:
#updata components
import os
import urllib.request as request
import zipfile
from chicken_disease_prediction import logger
from chicken_disease_prediction.utils.common import get_size

In [9]:
import gdown

In [ ]:
# check downloaded file exactly zip file or not
import zipfile

def is_zipfile_valid(filepath):
    return zipfile.is_zipfile(filepath)


In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename = gdown.download(
                url=self.config.source_url,
                output=self.config.local_data_file,
                quiet=False,
                fuzzy=True
            )
            logger.info(f"{filename} downloaded successfully")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file into the data directory
        function returns None
        """

        if zipfile.is_zipfile(self.config.local_data_file):
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
                logger.info(f"Extracted all files to {self.config.unzip_dir}")
        else:
            logger.error("Downloaded file is not a valid zip file.")
            raise zipfile.BadZipFile("Downloaded file is not a valid zip file.")
            

In [15]:
#create pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-05-19 17:19:29,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 17:19:29,623: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 17:19:29,624: INFO: common: created directory at: artifacts]
[2025-05-19 17:19:29,625: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?id=1pHWZX77G1Ue1pKq0m-lHCTukHrkX1FcP
From (redirected): https://drive.google.com/uc?id=1pHWZX77G1Ue1pKq0m-lHCTukHrkX1FcP&confirm=t&uuid=3b699053-a93a-4189-b7e9-fafc1fee340c
To: d:\myprojects\Chicken_disease_classification\Chicken-disease-classification\artifacts\data_ingestion\Dataset.zip
100%|██████████| 101M/101M [00:04<00:00, 21.4MB/s] 

[2025-05-19 17:19:39,051: INFO: 19171549: artifacts/data_ingestion/Dataset.zip downloaded successfully]


[2025-05-19 17:19:41,085: INFO: 19171549: Extracted all files to artifacts/data_ingestion]
